## Import Modules

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Read Data

In [17]:
time_df = pd.read_csv("/Users/james/Desktop/BD-Compartmental-Model/Data/Time.csv", encoding = "UTF-8")
time_df.index = pd.to_datetime(time_df["date"])
time_df.drop("date", axis = 1, inplace=True)

time_province_df = pd.read_csv("//Users/james/Desktop/BD-Compartmental-Model/Data/TimeProvince.csv", encoding = "UTF-8")
time_province_df.index = pd.to_datetime(time_province_df["date"])
time_province_df.drop("date", axis = 1, inplace = True)

In [23]:
time_df.tail(3)

,time,test,negative,confirmed,released,deceased
date,,,,,,
2020-04-18,0,554834,530631,10653,7937,232
2020-04-19,0,559109,536205,10661,8042,234
2020-04-20,0,563035,540380,10674,8114,236


            time    test  negative  confirmed  released  deceased
date                                                             
2020-04-16     0  538775    513894      10613      7757       229
2020-04-17     0  546463    521642      10635      7829       230
2020-04-18     0  554834    530631      10653      7937       232
2020-04-19     0  559109    536205      10661      8042       234
2020-04-20     0  563035    540380      10674      8114       236


In [35]:
# Split train and test on confirmed dataset
confirm_df = time_df[["confirmed"]]

# Predict 5 days afterward
lstm_train = confirm_df.iloc[: len(confirm_df) - 5]
lstm_test = confirm_df.iloc[len(confirm_df) - 5:]

In [37]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler

# Scale between 0-1
scaler = MinMaxScaler()
scaler.fit(lstm_train)
preprocessed_train = scaler.transform(lstm_train)
preprocessed_test = scaler.transform(lstm_test)

In [38]:
from keras.preprocessing.sequence import TimeseriesGenerator

n_features = 1
n_input = 5
generator = TimeseriesGenerator(preprocessed_train, preprocessed_train, length= n_input, batch_size = n_features)

In [39]:
# Must download tensorflow as well
from keras.models import Sequential
from keras.layers import Dense # connected neural network layer
from keras.layers import LSTM # Adding the Long Short-Term Memory layer
from keras.layers import Dropout # Prevents overfitting
from keras.layers import Activation

# grid search epochs, batch size
epochs = [10 * i for i in range(1, 11)]
batch_size = [1000, 5000] # add 5, 10, 20, 40, 60, 80, 100 etc
param_grid = dict(epochs = epochs, batch_size = batch_size)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

model = Sequential()
model.add(LSTM(150,activation = "relu", input_shape = (n_input, n_features)))
model.add(Dense(75, activation = 'relu'))
model.add(Dense(units = 1))
model.compile(optimizer = "adam",loss = "mse")